# Building a Multimodal Image Search App with Titan Embeddings and LangChain

This post demonstrates how to combine [Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html), [LangChain](https://python.langchain.com/docs/get_started/introduction) and [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/) to build a capable image search application. Titan's embeddings allow representing images and text in a common dense vector space, enabling natural language querying of images. FAISS provides a fast, scalable way to index and search those vectors. And LangChain offers abstractions to hook everything together and surface relevant image results based on a user's query.

By following the steps outlined, you'll be able to preprocess images, generate embeddings, load them into FAISS, and write a simple application that takes in a natural language query, searches the FAISS index, and returns the most semantically relevant images. It's a great example of the power of combining modern AI technologies to build applications.

![Diagram](data/build_images_vector_db.jpg)

## Prerequisites

Before we begin, make sure you have the following prerequisites set up:

1. **Install boto3:** This is the [AWS SDK for Python ](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingTheBotoAPI.html)that allows interacting with AWS services. Install with `pip install boto3`.
2. [Configure AWS credentials](https://docs.aws.amazon.com/braket/latest/developerguide/braket-using-boto3.html): Boto3 needs credentials to make API calls to AWS.
3. **Amazon Titan Multimodal Embeddings:** Follow the instructions in the [Amazon Bedrock User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) to access the Titan Multimodal Embeddings model.
4. **LangChain:** Install the LangChain library by running `pip install langchain`.
5. **FAISS:** Install the FAISS vector database by running `pip install faiss-cpu` faiss-cpu (or faiss-gpu if you have a GPU available). 

https://catalog.workshops.aws/building-with-amazon-bedrock/en-US/image-labs/bedrock-image-search


In [ ]:
#!brew install postgresql
#!pip install boto3
#!pip install langchain
#!pip install faiss-cpu
#!pip install Pillow
# or install requirements.txt

In [1]:
import os
import boto3
import json
import base64
from langchain_community.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings
from io import BytesIO
from PIL import Image

In [2]:
bedrock_client              = boto3.client("bedrock-runtime", region_name='us-east-1') 
boto3_bedrock               = boto3.client('bedrock', region_name='us-east-1')
default_embedding_dimmesion = os.environ.get("DEFAULT_EMBEDDING_DIMENSION", "1024")
model_id = os.environ.get("DEFAULT_MODEL_ID", "amazon.titan-embed-image-v1")

### Validate the connection
We can check the client works by trying out the list_foundation_models() method, which will tell us all the models available for us to use

In [ ]:
[models['modelId'] for models in boto3_bedrock.list_foundation_models()['modelSummaries']]

## Generating Embeddings for Images
The first step is to determine whether we will be processing text or images. We identify this using the `get_multimodal_vector` function which takes the input and utilizes the Amazon Titan model through the [InvokeModel](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) API from [Amazon Bedrock](https://aws.amazon.com/bedrock/) to generate a joint embedding vector for the image or text, as applicable.

In [3]:
#calls Bedrock to get a vector from either an image, text, or both
def get_multimodal_vector(input_image_base64=None, input_text=None):
    
    session = boto3.Session()

    bedrock = session.client(service_name='bedrock-runtime', region_name='us-east-1') #creates a Bedrock client
    
    request_body = {}
    
    if input_text:
        request_body["inputText"] = input_text
        
    if input_image_base64:
        request_body["inputImage"] = input_image_base64
    
    body = json.dumps(request_body)
    
    response = bedrock.invoke_model(
    	body=body, 
    	modelId=model_id,
        accept="application/json", 
    	contentType="application/json"

    )
    
    response_body = json.loads(response.get('body').read())
    
    embedding = response_body.get("embedding")
    
    return embedding

`get_vector_from_file` function takes an image file path, encodes the image to base64, generates an embedding vector using [Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html), and returns the vector - allowing images to be represented as vectors

In [4]:
#creates a vector from a file
def get_vector_from_file(file_path):
    with open(file_path, "rb") as image_file:
        input_image_base64 = base64.b64encode(image_file.read()).decode('utf8')
    
    vector = get_multimodal_vector(input_image_base64 = input_image_base64)
    
    return vector

Creates a list of (path, vector) tuples from a directory.

A sample of [Kaggle Animal Image Dataset (90 Different Animals)](https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals) is used in this app.

In [5]:
def get_image_vectors_from_directory(path_name):
    items = []
    sub_1 = os.listdir(path_name)
    for n in sub_1:
        if n.endswith('.jpg'):
            file_path = os.path.join(path_name,n)
            size_image(file_path)
            vector = get_vector_from_file(file_path)
            items.append((file_path, vector))
        else:
            for n_2 in os.listdir(path_name+"/"+n):
                if n_2.endswith('.jpg'):
                    file_path = os.path.join(path_name+"/"+n, n_2)
                    size_image(file_path)
                    vector = get_vector_from_file(file_path)
                    items.append((file_path, vector))
                else:
                    print("no a .jpg file: ",n_2)

    return items


In [6]:
def size_image(file_path):
    # Maximum image size supported is 2048 x 2048 pixel
    image = Image.open(file_path) #open image
    width, height = image.size # Get the width and height of the image in pixels
    if width > 2048 or height > 2048:
        print(f"Big File:{file_path} , width: {width}, height {height} px")
        dif_width = width - 2048
        dif_height = height - 2048
        if dif_width > dif_height:
            ave = 1-(dif_width/width)
            new_width = int(width*ave)
            new_height = int(height*ave)
        else:
            ave = 1-(dif_height/height)
            new_width = int(width*ave)
            new_height = int(height*ave)
        print(f"New file: {file_path} , width: {new_width}, height {new_height} px")

        new_image = image.resize((new_width, new_height))
        # Save New image
        new_image.save(file_path)
 
    return


### Creates and returns an in-memory vector store to be used in the application

In [7]:
model_kwargs  = {"embeddingConfig": { "outputEmbeddingLength": 1024}}
def get_bedrock_embeddings(model_id, model_kwargs):
    return BedrockEmbeddings(model_id=model_id, client=bedrock_client, model_kwargs=model_kwargs)

In [8]:
def create_vector_db(path_name):
    image_vectors = get_image_vectors_from_directory(path_name)
        
    text_embeddings = [("", item[1]) for item in image_vectors]
    metadatas = [{"image_path": item[0]} for item in image_vectors]
        
    db = FAISS.from_embeddings(
        text_embeddings=text_embeddings,
        embedding = get_bedrock_embeddings(model_id, model_kwargs),
        metadatas = metadatas
    )
    print(f"Vector Database:{db.index.ntotal} docs")
    return db

In [ ]:
path_file = "animals/animals"
path_name = f"{path_file}"
db = create_vector_db(path_name)

### [Save to a local Vector database.](https://python.langchain.com/docs/integrations/vectorstores/faiss/#as-a-retriever)

In [ ]:
db_file = "animals.vdb"
db.save_local(db_file)
print(f"vectordb was saved in {db_file}")

https://python.langchain.com/docs/how_to/#vector-stores

### [Query](https://python.langchain.com/docs/integrations/vectorstores/faiss/#querying) by text

In [11]:
query = "dog"
search_vector = get_multimodal_vector(input_text=query)
results = db.similarity_search_by_vector(embedding=search_vector)
for res in results:
    with open(res.metadata['image_path'], "rb") as f:
        img = BytesIO(f.read())
        image = Image.open(img)
        image.show()

### [Query](https://python.langchain.com/docs/integrations/vectorstores/faiss/#querying) by Image

In [12]:
query_path = 'animals/animals/cat/9d21019336.jpg'
vector = get_vector_from_file(query_path)
results = db.similarity_search_by_vector(embedding=vector)
for res in results:
    with open(res.metadata['image_path'], "rb") as f:
        img = BytesIO(f.read())
        image = Image.open(img)
        image.show()

### Load and Query local Vector database

In [13]:
from langchain_aws import BedrockEmbeddings  # to create embeddings for the documents.
db_file = "animals.vdb"
bedrock_embeddings          = BedrockEmbeddings(model_id=model_id,client=bedrock_client)
new_db = FAISS.load_local(db_file, bedrock_embeddings, allow_dangerous_deserialization=True)

In [14]:
query = "cat"
search_vector = get_multimodal_vector(input_text=query)
results = new_db.similarity_search_by_vector(embedding=search_vector)
for res in results:
    with open(res.metadata['image_path'], "rb") as f:
        img = BytesIO(f.read())
        image = Image.open(img)
        image.show()

In [24]:
query_path = 'animals/animals/leopard/9cc45df890.jpg'
vector = get_vector_from_file(query_path)
results = db.similarity_search_by_vector(embedding=vector)
for res in results:
    with open(res.metadata['image_path'], "rb") as f:
        img = BytesIO(f.read())
        image = Image.open(img)
        image.show()

In [ ]:
results

### [Delete Vectordb](https://python.langchain.com/docs/integrations/vectorstores/faiss/#delete)

You can also delete records from the vector store.

In [ ]:
print("count before:", new_db.index.ntotal)
new_db.delete([new_db.index_to_docstore_id[0]])
print("count after:", new_db.index.ntotal)

Delete the entire database

In [ ]:
new_db.delete